In [10]:
import numpy as np

from qiskit_qec.geometry.model.qubit_count import QubitCount
from qiskit_qec.geometry.model.qubit_data import QubitData
from qiskit_qec.operators.pauli import Pauli
from qiskit_qec.operators.pauli_list import PauliList
import qiskit_qec.utils.pauli_rep as rep

from qiskit_qec.geometry.tiles.RSSC import RSSC
from qiskit_qec.geometry.shape import Shape
from qiskit_qec.geometry.plane import Plane
from qiskit_qec.geometry.lattice import Lattice
from qiskit_qec.geometry.model.tiling import Tiling

In [2]:
# Set parameters for RSSC code on the plane
d=3
manifold = Plane()

# Create the bounding or cutter shape
P = Shape.square(origin=(0,-1), direction=(1,1), length=d-1, manifold=manifold, dtype=int)

# Create an extended bounding box AABB
Be = P.bounds.copy()
amount = np.array([2,2])/2
Be.expand(amount)

# Basis for Tiling
u = np.array([2,0])
v = np.array([0,2])

# Create Tiling lattice
L = Lattice(u,v)

# Calculate shear/transform
l = L.find_shear_length(Be.size)

# Create standard l x l lattice to be transformed 
S_l = Lattice(size=(l,l))

# Shear/transform the standard lattice into a sublattice of L
L_l = S_l.apply_transform_from(L)

# Mask the lattice with the expanded AABB
L_l = L_l.restrict(Be)

# Set up the qubit counter and aux data structure
qubit_count = QubitCount()
qubit_data = QubitData()

# Tile the restriced lattice L_l with RSSC tiles
M = Tiling(tile=RSSC, lattice=L_l, qubit_count=qubit_count, qubit_data=qubit_data)

# Determine the intersection of faces/operators with defined region P
operator_intersect = dict()
for face in M.faces:
    count = 0
    for vertex in face.vertices:
        if P.contains(vertex.pos):
            count += 1
    operator_intersect[face.id] = count

# Split faces/operators into intersection level categories
part = [[],[],[],[]]
for id in operator_intersect.keys():
    part[operator_intersect[id]].append(id)
 

In [2]:
   
# Cull faces.operators with k<2 qubits inside P

k=2
for i in range(k):
    # Update the auxillary data
    for face in M.faces:
        if face.id in part[i]:
            for vertex in face.vertices:
                qubit_id = qubit_data.qubit[vertex.id]
                qubit_count.decrement_qubit(qubit_id)
      
    # Delete faces and components from shell
    M.delete_subtree(part[i])
    



In [1]:
import numpy as np
from qiskit_qec.geometry.model.vertex import Vertex
from qiskit_qec.geometry.model.edge import Edge
from qiskit_qec.geometry.model.wireframe import WireFrame
from qiskit_qec.geometry.model.face import Face
from qiskit_qec.geometry.model.shell import Shell

In [2]:
 def make_test():
    a = Vertex(np.array([1,1]))
    b = Vertex(np.array([2,2]))
    c = Vertex(np.array([3,3]))

    e1 = Edge([a,b])
    e2 = Edge([b,c])

    wf = WireFrame([e1,e2])
    f = Face(wf)

    s = Shell([f])
    
    
    return s

In [3]:
s = make_test()

In [4]:
def del_test(s):
    tf = s.faces
    s = None
    for face in tf:
        twf = face.wireframe
        face = None
        for wireframe in [twf]:
            tedges = wireframe.edges
            wireframe = None
            for edge in tedges:
                tvertices = edge.vertices
                edge = None
                for vertex in tvertices:
                    vertex = None


In [6]:
del_test(s)